In [14]:
%pip install langchain-groq
from langchain_groq import ChatGroq


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key = "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",
    model_name = "openai/gpt-oss-20b"  
)

response = llm.invoke("Write a cold email for person looking for a job in data science.")
print(response.content)




**Subject:** Data Scientist Seeking Impact‑Driven Opportunities at [Company Name]

Hi [Hiring Manager’s Name],

I hope you’re doing well. I’m a data scientist with 3+ years of experience turning complex business problems into actionable insights. At my current role with **[Current Company]**, I led a cross‑functional team that built a predictive churn model that cut customer attrition by 18% and saved the company $2.3 M annually.

Key strengths I bring:

- **Advanced analytics & ML** – Python, R, SQL, scikit‑learn, TensorFlow; deployed models to production on AWS SageMaker.  
- **Data storytelling** – Tableau & Power BI dashboards that translate metrics into clear, executive‑level narratives.  
- **Domain expertise** – Finance & e‑commerce, with a proven track record of improving revenue and operational efficiency.  

I’m excited about the work [Company Name] is doing in **[specific area, e.g., AI‑driven personalization]** and would love to discuss how my background could help accelera

In [16]:
%pip install beautifulsoup4
%pip install langchain-community
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip






















Nike Careers









































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Langua

In [17]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [18]:

from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': '12 Months Head Coach (Senior Store Manager), Nike - Brisbane Airport',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Account Executive, SEA&IR',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Alternant (Vendeur en magasin)-H/F',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Analyst II, Origin Operations',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Assistant Head Coach (Directeur / Directrice Adjoint de magasin) – H/F',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Assistant Head Coach - Oxford Circus',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Assistant Head Coach- Dundrum',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Assistant Store Manager',
  'experience': '',
  'skills': '',
  'description': ''},
 {'role': 'Assistant Store Manager (Assistant Head Coach) - NIKE Lelystad',
  'experience': '',
  'skills': ''

In [19]:

type(json_res)

list

In [20]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [21]:
%pip install chromadb
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [22]:
for job in json_res:
	# Use job['role'] as query text if job['skills'] is None, otherwise use job['skills']
	query_text = job['skills'] if job['skills'] else job['role']
	links = collection.query(query_texts=query_text, n_results=2).get('metadatas', [])
	print(f"Job Role: {job['role']}")
	print(f"Relevant Links: {links}")
	print("-" * 30)

Job Role: 12 Months Head Coach (Senior Store Manager), Nike - Brisbane Airport
Relevant Links: [[{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/xamarin-portfolio'}]]
------------------------------
Job Role: Account Executive, SEA&IR
Relevant Links: [[{'links': 'https://example.com/ios-ar-portfolio'}, {'links': 'https://example.com/kotlin-backend-portfolio'}]]
------------------------------
Job Role: Alternant (Vendeur en magasin)-H/F
Relevant Links: [[{'links': 'https://example.com/kotlin-backend-portfolio'}, {'links': 'https://example.com/typescript-frontend-portfolio'}]]
------------------------------
Job Role: Analyst II, Origin Operations
Relevant Links: [[{'links': 'https://example.com/angular-portfolio'}, {'links': 'https://example.com/typescript-frontend-portfolio'}]]
------------------------------
Job Role: Assistant Head Coach (Directeur / Directrice Adjoint de magasin) – H/F
Relevant Links: [[{'links': 'https://example.com/typescript-fron

In [23]:
job

{'role': 'Assistant Store Manager (Assistant Head Coach) - NIKE Roermond',
 'experience': '',
 'skills': '',
 'description': ''}

In [24]:
job = json_res
job[0]['skills']

''

In [25]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Accelerate Your Retail Operations & Talent Management with AtliQ’s AI‑Driven Solutions  

Hi [Hiring Manager’s Name],

I hope you’re doing well. I’m Mohan, Business Development Executive at AtliQ, where we specialize in turning complex retail and talent‑management challenges into streamlined, data‑driven workflows.

**Why AtliQ is a perfect fit for your current openings**

| Role | Key Pain Points | How AtliQ Helps |
|------|-----------------|-----------------|
| **Head Coach / Assistant Head Coach / Assistant Store Manager** | Managing store performance, coaching staff, and aligning local initiatives with corporate strategy | • AI‑powered performance dashboards that track KPIs in real time<br>• Automated coaching plans that adapt to individual staff metrics<br>• Seamless integration with existing HR & POS systems |
| **Account Executive, SEA & IR** | Lead generation, pipeline visibility, and cross‑channel coordination | • Intelligent CRM workflows that surface high‑value pros